In [1]:
import pandas as pd
import regex as re
import numpy as np
from pandasql import sqldf

In [13]:
def win_conditions(row):
    if 'penalties' in row['Win Conditions']:
        return 'penalties'
    else:
        return row['Win Conditions']

In [14]:
def home_winner(row):
    # This functions will return if Spain either won or lost a match 
    if row['Home Goals'] > row['Away Goals']:
        return 'Win'
    elif row['Home Goals'] == row['Away Goals'] and row['Win Conditions'] == 'normal ending':
        return 'Draw'
    elif row['Home Goals'] == row['Away Goals'] and row['Home Team'] in row['Win Conditions']:
        return 'Win'
    else:
        return 'Loss'

In [15]:
def away_winner(row):
    if row['Away Goals'] > row['Home Goals']:
        return 'Win'
    elif row['Home Goals'] == row['Away Goals'] and row['Win Conditions'] == 'normal ending':
        return 'Draw'
    elif row['Home Goals'] == row['Away Goals'] and row['Away Team'] in row['Win Conditions']:
        return 'Win'
    else:
        return 'Loss'

In [16]:
def setting_history_matches(df):
    df['win_conditions'] =  df.apply(lambda row: win_conditions(row), axis=1)
    df['home_winner'] = df.apply(lambda row: home_winner(row), axis=1)
    df['away_winner'] = df.apply(lambda row: away_winner(row), axis=1)
    
    df = df.drop(columns={'Win Conditions'})
    
    return df

In [18]:
groups_22 = pd.read_csv('data\\2022_world_cup_groups.csv')

In [19]:
matches_22 = pd.read_csv('data\\2022_world_cup_matches.csv')

In [187]:
data_dict = pd.read_csv('data_dictionary.csv') ## -> fields description

In [33]:
international_matches = pd.read_csv('data\\international_matches.csv')
international_matches.drop(columns={'ID'}, inplace= True)
international_matches.isnull().sum() #Win conditions have null values when matches have ended normally
international_matches['Win Conditions'] = international_matches['Win Conditions'].fillna('normal ending')

In [34]:
international_matches.rename(columns={"Tournament":"Stage","Home Stadium":"Host Team"},inplace = True)

In [45]:
international_matches['Tournament']='International Matches'

In [49]:
col_list = list(international_matches)

In [50]:
international_matches = international_matches.reindex(columns=['Date','Stage','Home Team','Home Goals',
                                                               'Away Goals','Away Team','Win Conditions','Host Team','Tournament'])

In [2]:
world_cup_matches = pd.read_csv('data\\world_cup_matches.csv')

In [55]:
world_cup_matches = pd.read_csv('data\\world_cup_matches.csv')
world_cup_matches.drop(columns={'ID'}, inplace= True)
world_cup_matches.isnull().sum() #Win conditions have null values when matches have ended normally
world_cup_matches['Win Conditions'] = world_cup_matches['Win Conditions'].fillna('normal ending')

In [56]:
world_cup_matches.drop(columns={'Year'},inplace=True)

In [57]:
world_cup_matches['Tournament']='World Cup'

In [58]:
history_matches = pd.concat([international_matches, world_cup_matches], ignore_index=True)

In [199]:
def country_vs(df,team_A, team_B):
    df = df[((df['Home Team']==team_A) & (df['Away Team']==team_B))
                     | ((df['Away Team']==team_A) & (df['Home Team']==team_B))]
    return df
    

## Football History of Spain

In [59]:
final_df = setting_history_matches(history_matches)

In [61]:
final_df['match'] = final_df['Home Team']+ ' vs ' + final_df['Away Team']

In [62]:
home = final_df[final_df.columns.difference(['Away Team', 'Away Goals', 'away_winner'])]

In [63]:
away = final_df[final_df.columns.difference(['Home Team', 'Home Goals', 'home_winner'])]

In [64]:
home = home.loc[:, ['Date', 'Stage', 'Home Team', 'Home Goals', 'Host Team', 'home_winner', 
                    'win_conditions', 'match','Tournament']]
home.rename(columns={'Home Team':'Team', 'Home Goals':'Goals', 'home_winner':'Result'}, inplace=True)

In [65]:
away = away.loc[:, ['Date', 'Stage', 'Away Team', 'Away Goals', 'Host Team', 'away_winner',
                    'win_conditions', 'match','Tournament']]
away.rename(columns={'Away Team':'Team', 'Away Goals':'Goals', 'away_winner':'Result'}, inplace=True)

In [66]:
df = pd.concat([home,away], ignore_index=True)

In [67]:
merged_df = pd.merge(df, df, left_on=['Date','match'], right_on=['Date','match'])

In [68]:
final_result = merged_df[merged_df.Team_x != merged_df.Team_y]

In [69]:
x = final_result.loc[:,['Date', 'Stage_x', 'Team_x', 'Goals_x','Result_x','Team_y', 'Goals_y','win_conditions_x','Tournament_x']]

In [70]:
x = x.rename(columns ={'Stage_x':'Stage', 'Team_x':'Team', 'Goals_x':'Home Goals', 
                       'Goals_x':'Home Goals', 'Result_x':'Result', 'Team_y':'Team Against',
                      'Goals_y':'Goals Against','win_conditions_x':'Win Conditions','Tournament_x':'Tournament'})

In [351]:
x.to_csv('C:\\Users\\juanp\\proyectos_varios\\world_cup\\history_matches.csv')

In [142]:
teams_22 = ['Qatar', 'Ecuador', 'Senegal', 'Netherlands',
            'England', 'Iran','United States','Wales',
           'Argentina','Saudi Arabia','Mexico','Poland',
           'France','Australia','Denmark','Tunisia',
           'Spain', 'Costa Rica','Germany','Japan',
            'Belgium','Canada', 'Morocco','Croatia',
            'Brazil', 'Serbia', 'Switzerland', 'Cameroon',
            'Portugal', 'Ghana', 'Uruguay', 'South Korea']

In [227]:
teams_world_cup_22 = x[(x['Team'].isin(teams_22)) & (x['Team Against'].isin(teams_22))]

# Analyzing Spains' results 

In [352]:
spain = x[x.Team == 'Spain']

In [275]:
def results(df):
    df_results = df.groupby([
    'Team Against','Result','Tournament','Stage']).agg({'Result':'count'}).rename(columns={'Result':'Matches'}).reset_index()
    df_results = df_results.pivot_table('Matches',['Team Against','Tournament','Stage'],'Result', fill_value=0).reset_index()
    return df_results

In [128]:
# s = spain_results.div(spain_results.sum(axis=1),axis=0) --> maybe I can obtain the same result directly in tableau and keep the total

# How was Spain's performance in previous World Cups?

In [161]:
a = list(world_cup_matches['Stage'].unique())

In [353]:
spain_international =spain[~spain['Stage'].isin(a)]

In [354]:
spain_wc = spain[spain['Stage'].isin(a)]

In [355]:
spain_complete = pd.concat([spain_wc, spain_international],ignore_index=True)

In [281]:
spain_matches.to_csv('C:\\Users\\juanp\\proyectos_varios\\world_cup\\spain_matches.csv')

In [93]:
x = pd.read_csv('data\\spain_matches.csv')

In [94]:
x = x.drop(columns={'Unnamed: 0'})

In [95]:
x = x.groupby('Team Against').agg({'Draw':'sum','Win':'sum','Loss':'sum'})

In [97]:
x['Matches'] = x.sum(axis=1)

In [99]:
x.to_csv('C:\\Users\\juanp\\proyectos_varios\\world_cup\\spain_matches_totals.csv')

In [6]:
x = world_cup_matches 

In [50]:
s = pd.read_csv('data\\history_matches.csv')

In [51]:
d = pd.read_csv('data\\2022_world_cup_groups.csv')

In [88]:
teams = pd.merge(s,d,left_on='Team', right_on='Team' )

In [181]:
teams['Stage_value'] = np.where(teams['Stage']=='Quarter-finals',3,0)

In [182]:
teams_ranking = teams.groupby(['Team','Result','FIFA Ranking']).agg({'Result':'count'})

In [183]:
teams_ranking.rename(columns={'Result':'Results'},inplace=True)

In [184]:
teams_ranking = teams_ranking.reset_index()

In [138]:
conditions = [
    (teams_ranking['FIFA Ranking'] <= 10),
    (teams_ranking['FIFA Ranking'] > 10) & (teams_ranking['FIFA Ranking'] <= 20),
    (teams_ranking['FIFA Ranking'] > 20) & (teams_ranking['FIFA Ranking'] <= 40),
    (teams_ranking['FIFA Ranking'] > 40),
]

values = [4,3,2,1]

In [139]:
teams_ranking['Ranking_Value'] = np.select(conditions, values)

In [140]:
total_matches = teams.groupby('Team').agg({'Result':'count'})

In [185]:
quarters_reacher = teams[['Team','Stage_value']].drop_duplicates(keep='first')

In [154]:
df = [teams_ranking, total_matches, quarters_reacher]

In [189]:
teams_ranking['Quarters_reacher'] = teams_ranking.Team.map(quarters_reacher.set_index('Team')['Stage_value'].to_dict())

In [195]:
teams_ranking['Matches'] = teams_ranking.Team.map(total_matches['Result'].to_dict())

In [203]:
teams_ranking.to_csv('C:\\Users\\juanp\\proyectos_varios\\world_cup\\teams_rankings.csv')

In [204]:
teams_ranking

,Team,Result,FIFA Ranking,Results,Matches,Ranking_value
0,Argentina,Draw,3,233,1010,3
1,Argentina,Loss,3,219,1010,3
2,Argentina,Win,3,558,1010,3
3,Australia,Draw,38,110,536,0
4,Australia,Loss,38,154,536,0
...,...,...,...,...,...,...
91,Uruguay,Loss,14,301,916,3
92,Uruguay,Win,14,406,916,3
93,Wales,Draw,19,151,689,3
94,Wales,Loss,19,317,689,3
